### This notebook has the preliminary runs to get the working querries, which are not in functional form.
The script analysis_frontend.py run the menu and calls the up to date functions (from functions.py) 

In [7]:
import os
import re
import sys

In [8]:
# add path to the the main directory
sys.path.append(os.getcwd()+'/..')  

In [9]:

from pyspark.sql import SparkSession
import mysql.connector
from pyspark.sql.functions import regexp_replace, regexp_extract, col
from functions import (
    transaction_by_customer,
    get_val_trans_type,
    get_trans_state,
    get_cust_info,
    modify_cust_record,
    mysql_updater,
    generate_bill,
    get_trans_interval,
)


In [10]:
spark = SparkSession\
    .builder\
    .appName("capstone_analysis")\
    .config("spark.jars","/opt/homebrew/Cellar/apache-spark/3.3.1/libexec/jars/mysql-connector-j-8.0.32.jar")\
    .getOrCreate()

In [11]:
mysql_pwd = os.environ.get("mysql_pwd")
mysql_user = os.environ.get("mysql_user")

### Read data from sources


In [ ]:
# read tables

credit = spark.read \
    .jdbc("jdbc:mysql://localhost:3306/creditcard_capstone","CDW_SAPP_CREDIT_CARD", \
          properties={"user": mysql_user, "password": mysql_pwd, "driver":"com.mysql.cj.jdbc.Driver"})
credit.createTempView("credit_tb")
credit.show(3)

In [ ]:
branch = spark.read \
    .jdbc("jdbc:mysql://localhost:3306/creditcard_capstone","CDW_SAPP_BRANCH", \
          properties={"user": mysql_user, "password": mysql_pwd,  "driver":"com.mysql.cj.jdbc.Driver"})
branch.createTempView("branch_tb")
branch.show(3)

+-----------+------------+-----------------+-----------------+------------+----------+-------------+-------------------+
|BRANCH_CODE| BRANCH_NAME|    BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP| BRANCH_PHONE|       LAST_UPDATED|
+-----------+------------+-----------------+-----------------+------------+----------+-------------+-------------------+
|          1|Example Bank|     Bridle Court|        Lakeville|          MN|     55044|(123)456-5276|2018-04-18 16:51:47|
|          2|Example Bank|Washington Street|          Huntley|          IL|     60142|(123)461-8993|2018-04-18 16:51:47|
|          3|Example Bank|    Warren Street|SouthRichmondHill|          NY|     11419|(123)498-5926|2018-04-18 16:51:47|
+-----------+------------+-----------------+-----------------+------------+----------+-------------+-------------------+
only showing top 3 rows



In [ ]:
customer = spark.read \
    .jdbc("jdbc:mysql://localhost:3306/creditcard_capstone","CDW_SAPP_CUSTOMER", \
          properties={"user":mysql_user,  "password": mysql_pwd,"driver":"com.mysql.cj.jdbc.Driver"})
customer.createTempView("customer_tb")
customer.show(3)

+---------+----------+-----------+---------+----------------+--------------------+------------+----------+-------------+--------+-------------+-------------------+-------------------+
|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  CREDIT_CARD_NO| FULL_STREET_ADDRESS|   CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|   CUST_PHONE|         CUST_EMAIL|       LAST_UPDATED|
+---------+----------+-----------+---------+----------------+--------------------+------------+----------+-------------+--------+-------------+-------------------+-------------------+
|123456100|      Alec|         wm|   Hooper|4210653310061055|Main Street North...|     Natchez|        MS|United States|   39120|(111)123-7818|AHooper@example.com|2018-04-21 12:49:02|
|123453023|      Etta|    brendan|   Holman|4210653310102868|  Redwood Drive, 829|Wethersfield|        CT|United States|    6109|(111)123-8933|EHolman@example.com|2018-04-21 12:49:02|
|123454487|    Wilber|   ezequiel|   Dunham|4210653310116272|12th Street East,..

In [ ]:
customer.printSchema()

root
 |-- SSN: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- FULL_STREET_ADDRESS: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_ZIP: integer (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)



### REQ-2.1: Transaction Details 
1)    Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in descending order.
2)    Used to display the number and total values of transactions for a given type.
3)    Used to display the number and total values of transactions for branches in a given state.

In [ ]:
# get inputs (write input function with argparse)
zipcode = 39120
month = str(2)
year = str(2018)
date_string = year+month.zfill(2)
date_string

'201802'

In [ ]:
q = "SELECT credit_tb.TRANSACTION_TYPE, "\
    "credit_tb.TRANSACTION_VALUE, "\
    "credit_tb.TRANSACTION_ID, "\
    "credit_tb.TIMEID, "\
    "customer_tb.CUST_ZIP "\
    "FROM credit_tb " \
    "JOIN customer_tb " \
    "ON credit_tb.CUST_SSN = customer_tb.SSN "\
    f"WHERE customer_tb.CUST_ZIP = {zipcode} "\
    f"AND credit_tb.TIMEID LIKE '{date_string}%' "\
    "ORDER BY TIMEID ASC;"

spark.sql(q).show()

+----------------+-----------------+--------------+--------+--------+
|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|  TIMEID|CUST_ZIP|
+----------------+-----------------+--------------+--------+--------+
|             Gas|            89.75|         22989|20180201|   39120|
|            Test|            55.19|          2211|20180201|   39120|
|             Gas|            40.91|          3530|20180202|   39120|
|       Education|            28.96|         15206|20180203|   39120|
|            Test|            38.85|          3548|20180204|   39120|
|      Healthcare|             38.4|          3549|20180205|   39120|
|       Education|            58.35|         12643|20180206|   39120|
|      Healthcare|            25.24|          2196|20180206|   39120|
|       Education|            17.88|          2216|20180207|   39120|
|      Healthcare|            46.21|         20550|20180208|   39120|
|           Bills|            22.17|         15267|20180209|   39120|
|            Test|  

In [ ]:
#2. given transaction types:
spark.sql("SELECT DISTINCT(TRANSACTION_TYPE) FROM credit_tb;").show()

+----------------+
|TRANSACTION_TYPE|
+----------------+
|       Education|
|   Entertainment|
|      Healthcare|
|         Grocery|
|            Test|
|             Gas|
|           Bills|
+----------------+



In [ ]:
trans_type = "Education"
spark.sql(f"SELECT TRANSACTION_TYPE, SUM(TRANSACTION_VALUE) FROM credit_tb GROUP BY TRANSACTION_TYPE HAVING TRANSACTION_TYPE = '{trans_type}';").show()

+----------------+----------------------+
|TRANSACTION_TYPE|sum(TRANSACTION_VALUE)|
+----------------+----------------------+
|       Education|    337980.07000000146|
+----------------+----------------------+



In [ ]:
spark.sql("SELECT DISTINCT(BRANCH_STATE) FROM branch_tb;").show()


+------------+
|BRANCH_STATE|
+------------+
|          SC|
|          MN|
|          NJ|
|          VA|
|          KY|
|          MI|
|          WI|
|          CA|
|          CT|
|          MT|
|          NC|
|          MD|
|          IL|
|          WA|
|          MS|
|          AL|
|          IN|
|          OH|
|          IA|
|          PA|
+------------+
only showing top 20 rows



In [ ]:
state = 'WI'
q = "SELECT branch_tb.BRANCH_STATE, "\
    "COUNT(TRANSACTION_ID), "\
    "SUM(TRANSACTION_VALUE) "\
    "from credit_tb JOIN branch_tb ON credit_tb.BRANCH_CODE = branch_tb.BRANCH_CODE "\
    f"WHERE branch_tb.BRANCH_STATE = '{state}'"\
    "GROUP BY branch_tb.BRANCH_STATE;"
spark.sql(q).show()

+------------+---------------------+----------------------+
|BRANCH_STATE|count(TRANSACTION_ID)|sum(TRANSACTION_VALUE)|
+------------+---------------------+----------------------+
|          WI|                 1677|     85899.84000000005|
+------------+---------------------+----------------------+



### REQ-2.2: Customer Details Module
1) Used to check the existing account details of a customer. (hide part of ssn)
2) Used to modify the existing account details of a customer.
3) Used to generate a monthly bill for a credit card number for a given month and year.
4) Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order.


In [ ]:
cut_name = "Alec wm Hooper\n".strip("\n").split()
first = cut_name[0].title()
middle = cut_name[1].lower()
last = cut_name[2].title()
print(first, middle, last)
q = f"SELECT * FROM customer_tb WHERE FIRST_NAME='{first}' AND MIDDLE_NAME='{middle}' AND LAST_NAME='{last}'" 
dfnew  = spark.sql(q)
dfnew = dfnew.withColumn("SSN_masked", regexp_replace(dfnew['SSN'],r"(\d{5})(\d{4})", "*****$2"))
dfnew.select('SSN_masked','FIRST_NAME', 'MIDDLE_NAME', 'LAST_NAME', 
                       'CREDIT_CARD_NO', 'FULL_STREET_ADDRESS', 'CUST_CITY',
                        'CUST_STATE', 'CUST_COUNTRY', 'CUST_ZIP', 'CUST_PHONE', 
                        'CUST_EMAIL', 'LAST_UPDATED').show()

Alec wm Hooper
+----------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+-------------+-------------------+-------------------+
|SSN_masked|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  CREDIT_CARD_NO| FULL_STREET_ADDRESS|CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|   CUST_PHONE|         CUST_EMAIL|       LAST_UPDATED|
+----------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+-------------+-------------------+-------------------+
| *****6100|      Alec|         wm|   Hooper|4210653310061055|Main Street North...|  Natchez|        MS|United States|   39120|(111)123-7818|AHooper@example.com|2018-04-21 12:49:02|
+----------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+-------------+-------------------+-------------------+



In [ ]:
# update table are not available error for the below expression (UPDATE TABLE is not supported temporarily)
spark.sql("UPDATE customer_tb  SET LAST_UPDATED = CURRENT_TIMESTAMP() WHERE FIRST_NAME='Alec'")


In [ ]:
# modify account details: (write out to the db, update last updated with timestamp now())
# read new info
# update table 
#save df to database
# Please enter item numbers to be modified:

def modify_customer_record():
    string1="""Please choose a number 1 to 5 to update one of the following:
    1 |-- NAME
    2 |-- CREDIT_CARD_NO
    3 |-- ADRESS
    4 |-- CUST_PHONE
    5 |-- CUST_EMAIL
    """
    choice = int(input(string1))
    if choice not in range(1,6):
        print('You entered a wrong choice!')
        choice = int(input(string1))

    ssn = input("Please enter the SSN for the record you wish to update").strip()
    if choice == 1:
        new_first = input("Enter new first name:").strip().title()
        new_middle = input("Enter new middle name:").strip().lower()
        new_last = input("Enter new last name:").strip().title()
        changes = f"UPDATE customer_tb SET FIRST_NAME = '{new_first}', "\
            f"MIDDLE_NAME = '{new_middle}', "\
            f"LAST_NAME = '{new_last}' "    

    elif choice == 2:
        new_card = input("Enter 16-digit new credit card number:").strip()
        changes = f"UPDATE customer_tb SET CREDIT_CARD_NO = '{new_card}'"

    elif choice == 3:
        new_add = input("""Enter street name, street number, 
        city, state, country, zipcode as a comma separated list""").strip().split(',')
        changes = f"UPDATE customer_tb SET FULL_STREET_ADRESS = '{new_add[0]}, {new_add[1]}', "\
            f"CUST_CITY = '{new_add[2]}', "\
            f"CUST_STATE = '{new_add[3]}', "\
            f"CUST_COUNTRY = '{new_add[4]}', "\
            f"ZIP_CODE = '{new_add[5]}' "

    elif choice == 4:
        new_phone = input("Enter new phone as a 10 digit number (no dashes,spaces or parantheses):").strip()
        new_phone = re.sub(r'(\d{3})(\d{3})(\d{4})',r"(\1)\2-\3", new_phone)
        changes = f"UPDATE customer_tb SET CUST_PHONE = '{new_phone}' "
    
    elif choice == 5:
        new_email = input("Enter new email address:").strip()
        changes = f"UPDATE customer_tb SET CUST_EMAIL = '{new_email}' "

    changes = changes + f"WHERE SSN = '{ssn}';"
    
    return ssn, changes

In [ ]:
ssn='123456100'
ssn, sql_update = modify_customer_record()


In [ ]:
sql_update

"UPDATE customer_tb SET FIRST_NAME = 'Alex', MIDDLE_NAME = 'd', LAST_NAME = 'Cooper' WHERE SSN = '123456100';"

In [ ]:
sql_update = "UPDATE CDW_SAPP_CUSTOMER SET CUST_PHONE = '(111)111-1111' WHERE SSN = '123456100';"

In [ ]:
ssn, sql_update
mydb = mysql.connector.connect(
  host="localhost",
  user=mysql_user,
  password=mysql_pwd,
  database="creditcard_capstone"
)

In [ ]:
mycursor = mydb.cursor()
sql = sql_update
mycursor.execute(sql)
mydb.commit()
print(mycursor.rowcount, "record(s) affected")

1 record(s) affected


In [ ]:
sql_update

"UPDATE CDW_SAPP_CUSTOMER SET CUST_PHONE = '(111)111-1111' WHERE SSN = '123456100';"

In [ ]:
import re
re.sub(r'(\d{3})(\d{3})(\d{4})',r"(\1)\2-\3", '1111111111' )

'(111)111-1111'

In [ ]:
credit.show(3)

+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|      CUST_CC_NO|  TIMEID| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|4210653349028689|20180214|123459988|        114|       Education|             78.9|             1|
|4210653312478046|20180813|123455692|        156|         Grocery|            91.08|         22562|
|4210653342242023|20180315|123451310|        180|           Bills|            77.79|         45069|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+
only showing top 3 rows



In [ ]:
month = str(2)
year = str(2018)
date_string = year+month.zfill(2)
credit_card_no = '4210653349028689'
date_string

'201802'

In [ ]:
q = "SELECT CUST_CC_NO, TIMEID, SUM(TRANSACTION_VALUE) as Daily_Total "\
    "FROM credit_tb " \
    f"WHERE CUST_CC_NO = '{credit_card_no}' "\
    f"AND TIMEID LIKE '{date_string}%' "\
    "GROUP BY CUST_CC_NO, TIMEID;"

spark.sql(q).show()

+----------------+--------+-----------+
|      CUST_CC_NO|  TIMEID|Daily_Total|
+----------------+--------+-----------+
|4210653349028689|20180203|      31.87|
|4210653349028689|20180224|      97.98|
|4210653349028689|20180206|      12.43|
|4210653349028689|20180215|      46.53|
|4210653349028689|20180202|       91.2|
|4210653349028689|20180214|       78.9|
+----------------+--------+-----------+



23/02/24 07:41:32 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 989018 ms exceeds timeout 120000 ms
23/02/24 07:41:32 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
q = "SELECT * FROM credit_tb WHERE CUST_CC_NO='4210653383230786' AND TIMEID between '20181101' and '20181231' order by TIMEID desc;"
spark.sql(q).show()

+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|      CUST_CC_NO|  TIMEID| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|4210653383230786|20181227|123459499|        188|           Bills|            88.71|          2574|
|4210653383230786|20181211|123459499|        126|   Entertainment|            14.58|          2606|
|4210653383230786|20181206|123459499|         58|       Education|            40.76|          2542|
|4210653383230786|20181205|123459499|         76|         Grocery|            98.46|          2616|
|4210653383230786|20181201|123459499|         90|            Test|            12.27|          2555|
|4210653383230786|20181118|123459499|         81|             Gas|             35.0|          2618|
|4210653383230786|20181116|123459499|        117|       Education|             87.6|          2629|


In [ ]:
spark.sql("SELECT TIMEID from credit_tb WHERE TIMEID between '20181201' and '20181203' ").show(30)

+--------+
|  TIMEID|
+--------+
|20181201|
|20181202|
|20181202|
|20181203|
|20181201|
|20181202|
|20181201|
|20181201|
|20181201|
|20181203|
|20181202|
|20181203|
|20181202|
|20181202|
|20181202|
|20181201|
|20181202|
|20181202|
|20181203|
|20181203|
|20181203|
|20181202|
|20181203|
|20181203|
|20181203|
|20181202|
|20181203|
|20181201|
|20181203|
|20181201|
+--------+
only showing top 30 rows



In [ ]:
spark.stop()